# Final Game
Hello!
For this final assigment, I've thinking about many games, that I can program, but, example game interests me, and recalling old ASCII games and Dwarf Fortress, I've tried to build something myself
So, I've decided to program Dungeon Crawler Game, with some mechanics (**exploring, search of items, randomly generated maps, fight mode**)
*Fair warning!* Full functionality was checked in Windows Command Prompt. I've adapted program for running in Ipython kernel, but some problems can occur. In addition to main *.ipynb* I've added .py file and .bat file, for easy launch

So, let's start!
First, I've imported *random* library and *os* module:

In [1]:
import random
import os

*Random* module is necessary here, because game use random functions to build dungeons and some calculations during fight

In [2]:
def cls_deprecated():
    os.system('cls' if os.name == 'nt' else 'clear')

from IPython.display import clear_output

def cls():
    clear_output(wait=True)

*cls_deprecated* meant to clear console output, so our map is updated, rather than printed over and over again
For Jupyter Notebook, we shall use *Ipython.display.cleat_output* function, so in the cell, we can clear the result, but that may works odd

In [3]:
WEAPONS = {'fists': {'attack': 2,
                     'level': 1},
           'knife': {'attack': 3,
                     'level': 3},
           'sword': {'attack': 6,
                     'level': 4},
           'axe': {'attack': 5,
                   'level': 4},
           'monster_weapon': {'attack': 3,
                              'level': 1}}

WEAPONS_ICONS = {'sword':
                     ['     #             ',
                      'O%%%%#============--       sword',
                      '     #               '
                      ],
                 'knife': ['[]++++||=======>    knife'],
                 'axe': [
                     '             +-+',
                     '=============| |      axe',
                     '            `:_;'],
                 'fists': ['fists']}

GRID_SYMBOLS = {'knife': '/',
                'sword': "+",
                'axe': 'c',
                'key': 'k',
                'fists': None}

MOVE_COMMANDS = {'w': (-1, 0),
                 's': (1, 0),
                 'd': (0, 1),
                 'a': (0, -1)}

GRID = [['#', '#', '#', '#', '#', '.', '.', '.', '#', 'x'],
        ['#', '#', '#', '.', '.', '.', '#', '.', '#', '.'],
        ['#', '#', '#', '.', '#', '#', '#', '.', '.', '.'],
        ['#', '#', '#', '.', '.', '.', '.', '#', '#', '#'],
        ['#', '#', '#', '#', '.', '.', '.', '#', '.', '#'],
        ['#', '#', '#', '#', '.', '.', '.', '#', '.', '#'],
        ['#', '#', '#', '.', '.', '.', '.', '#', '.', '#'],
        ['#', '#', '.', '.', '#', '#', '.', '.', '.', '#'],
        ['#', '#', '.', '#', '#', '#', '#', '#', '.', '.'],
        ['@', '.', '.', '#', '#', '#', '#', '#', '#', '#']]

MONSTER_NAMES = ['Big HSE Guy', 'Combination of Permutation', 'None', 'Lord of Variables', 'Rick Decart']

MONSTER_IMAGE = """
                 ||
                 ||
   ____ (((+))) _||_
  /.--.\  .-.  /.||.
  /.,   \(0.0)// ||
 /;`";/\ \|m|//  ||  ;
|:   \ \__`:`____||__:|
|:    \__ \T/ (@~)(~@)|
|:    _/|     |\_\/  :|
|:   /  |     |  \   :|
|'  /   |     |   \  '|
 \_/    |     |    \_/
        |     |
        |_____|
        |_____|
"""

EIGHT_DIRECTIONS = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

Here, we define some of our constants:
- WEAPONS: in this dictionary, I've defined weapons stats (level stat is deprecated)
- WEAPONS_ICONS: dictionary of ASCII art of weapons, that are showed in inventory screen
- GRID_SYMBOLS: symbols, used to define key items: **'/' - knife, '+' - sword, 'c' - axe, 'k' - key**
- MOVE_COMMANDS: symbols of moves, associated to move tuple, on grid: **'w' - up, 's' - down, 'a' - left, 'd' - right**
- GRID: predefined dungeon, that can be used. As you can see, '.' symbol shows where player **can walk**, '#' - **wall**, '@' - **player positions**, 'x' - **exit**
- MONSTER_NAMES: some names, for the monsters you'd meet (choosed randomly)
- MONSTER_IMAGE: ASCII art of monster, used in the games
- EIGHT_DIRECTIONS: directions, used to go on grid array for making visible cells on distance of 1 cell

The main goal of the game, is to find key(**'k'**) and find the exit(**'x'**). On your way, you can collect different weapons and meet monsters, which you have to fight!

In [4]:
class Grid:
    def __init__(self, width, height, monsters_number=3):
        self.grid = [[None for _ in range(width)] for _ in range(height)]
        self.width = width
        self.height = height
        self.start_coordinates = []
        self.end_coordinates = []
        self.monsters_number = monsters_number
        self.monsters = []
        self.path_cells = []

    def draw_grid(self):
        cls()
        for x in range(self.width):
            for y in range(self.height):
                print('%%-%ds' % 2 % self.grid[x][y][0], end='') if self.grid[x][y][1] else \
                    print('%%-%ds' % 2 % '.', end='')
            print()

    def get_allowed_cells(self):
        return_list = []
        for row in range(len(self.grid)):
            for col in range(len(self.grid[0])):
                if self.grid[row][col][0] == '.':
                    return_list.append([row, col])
        return return_list

    def set_predefined_grid(self, grid):
        self.grid = [[(grid[row][col], 0) for col in range(len(grid[0]))] for row in range(len(grid))]
        for row in range(len(self.grid)):
            for col in range(len(self.grid[0])):
                if grid[row][col][0] == 'x':
                    self.end_coordinates = [row, col]
                elif grid[row][col][0] == '@':
                    self.start_coordinates = [row, col]

    def place_items(self, items):
        allowed_cells = self.get_allowed_cells()
        for item in items:
            choosed_cell = random.choice(allowed_cells)
            allowed_cells.remove(choosed_cell)
            item.set_coordinates(choosed_cell)
            self.grid[choosed_cell[0]][choosed_cell[1]] = [item.get_symbol(), 0]

    def place_monsters(self):
        allowed_cells = self.get_allowed_cells()
        for _ in range(self.monsters_number):
            new_monster = Entity(name=random.choice(MONSTER_NAMES),
                                 coordinates=random.choice(allowed_cells),
                                 hitpoints=random.choice(range(10, 31)),
                                 attack=random.choice(range(1, 4)),
                                 defence=random.choice(range(1, 5)))
            allowed_cells.remove(new_monster.get_coordinates())
            new_monster.add_weapon(Weapon('monster_weapon'))
            self.monsters.append(new_monster)

    def isValid(self, row, col):
        return 0 <= row < len(self.grid) and 0 <= col < len(self.grid[0])

    def random_generator(self):
        """
        In the final step, I thought about, maybe some potential player
        wants to play some random levels, with own size of it
        So, my idea, is that we can build random walls with p = 1/2 (or less)
        and on ready example see, that there is existing path between start and finish
        I've thinking about DFS ot BFS attempt on this, but actually 've used DFS
        :return:
        """
        for row in range(self.height):
            for col in range(self.width):
                if random.random() < 0.8:
                    self.grid[row][col] = ['.', 0]
                else:
                    self.grid[row][col] = ["#", 0]

        allowed_cells = self.get_allowed_cells()
        self.start_coordinates = random.choice(allowed_cells)
        self.grid[self.start_coordinates[0]][self.start_coordinates[1]] = ['@', 0]

        allowed_cells.remove(self.start_coordinates)
        self.end_coordinates = random.choice(allowed_cells)
        self.grid[self.end_coordinates[0]][self.end_coordinates[1]] = ['x', 0]
        allowed_cells.remove(self.end_coordinates)

        queue = [self.start_coordinates]
        visited_grid = [[0 for _ in range(self.width)] for _ in range(self.height)]
        path_found = False
        while queue:
            cell = queue.pop()
            self.path_cells.append(cell)
            if cell == self.end_coordinates:
                print('Grid generated\n')
                input('Press Enter to continue... ')
                path_found = True
            for x, y in (0, 1), (1, 0), (0, -1), (-1, 0):
                if self.isValid(new_x := self.start_coordinates[0] + x, new_y := self.start_coordinates[1] + y) and \
                        not visited_grid[new_x][new_y] and self.grid[row][col][0] != '#':
                    queue.append([new_x, new_y])
                    visited_grid[new_x][new_y] = 1
        print('Re-generating grid\nPlease wait...')
        if not path_found:
            self.path_cells = []
            self.random_generator()

    def update_player(self, direction, new_coordinates):
        self.grid[new_coordinates[0] - direction[0]][new_coordinates[1] - direction[1]] = ['.', 1]
        self.grid[new_coordinates[0]][new_coordinates[1]] = ['@', 1]

    def get_element_symbol(self, coordinates):
        return self.grid[coordinates[0]][coordinates[1]][0]

    def get_monsters_list(self):
        return self.monsters

    def remove_monster(self, monster):
        self.monsters.remove(monster)

    def get_start_coordinates(self):
        return self.start_coordinates

    def make_cells_visible(self, player_coordinates):
        for x, y in EIGHT_DIRECTIONS:
            if self.isValid(new_x := player_coordinates[0] + x, new_y := player_coordinates[1] + y):
                self.grid[new_x][new_y][1] = 1


Up here, is the Grid object, which is used to store and display cells (walls, items, e.g.)
This object have fields with *width, height of game board; start and end coordinates, number of monsters, list of monster objects and cells, we can use, to move player
As, we can see, this grid can be generated randomly (using *random_generator()*), but we can pre-define shape of game board. Actually game board tracks coordinates of items, player and monsters, and use *draw_grid()* method to draw it as console output
For better understanding of code, I've tried to use principle, where each variable and function have clear name, so that the functionality of it is clear (only docstring is random_generator, because of BFS algo)

In [5]:
class Entity:
    def __init__(self, name, coordinates, hitpoints, attack=3, defence=1):
        self.name = name
        self.level = 1
        self.inventory = []
        self.hitpoints = hitpoints
        self.current_weapon = 0
        self.attack, self.defence = attack, defence
        self.coordinates = coordinates

    def base_attack(self):
        return round(random.uniform(0.7, 2) * (self.attack + self.inventory[self.current_weapon].base_attack()))

    def attack_entity(self, entity):
        entity.hitpoints -= self.base_attack()

    def get_coordinates(self):
        return self.coordinates

    def add_weapon(self, weapon):
        self.inventory.append(weapon)

    def get_hit(self, hit):
        self.hitpoints -= (hit - self.defence) if hit - self.defence >= 0 else self.hitpoints

    def get_defence(self):
        return self.defence

    def get_health(self):
        return self.hitpoints

    def levelup(self):
        self.level += 1

    def get_name(self):
        return self.name

**Entity** class defines base fields and methods of 2 base creatures: *monsters and player*
Here, we track self coordinates, health, inventory and current weapon, and also perform hit on entity and attack of entity to other entity
Memorable here is *base_attack()* method, which counts attack points of entity, based on self attack parameter plus weapon attack parameter with some random effect (perhaps your first kick weaker that second)

In [6]:
class Player(Entity):
    def move(self, direction):
        self.coordinates[0] += direction[0]
        self.coordinates[1] += direction[1]

    def collect_item(self, item):
        self.inventory.append(item)

    def select_weapon(self):
        self.current_weapon = (1 + self.current_weapon) % len(self.inventory)
        if self.inventory[self.current_weapon].get_name() == 'key':
            self.select_weapon()

    def draw_inventory(self):
        print()
        for item_index in range(len(self.inventory)):
            if (weapon_name := self.inventory[item_index].get_name()) != 'key':
                if self.current_weapon == item_index:
                    marker = len(WEAPONS_ICONS[weapon_name]) // 2
                    for line_index in range(len(weapon_icon := WEAPONS_ICONS[weapon_name])):
                        if line_index == marker:
                            print(weapon_icon[line_index], '    ', '<<<')
                        else:
                            print(weapon_icon[line_index])
                else:
                    for line in WEAPONS_ICONS[weapon_name]:
                        print(line)
            print()

    def has_key(self):
        for item in self.inventory:
            if item.get_symbol() == GRID_SYMBOLS['key']:
                return True

**Player** class if one of examples, of object *inheritance*. For out class Player, we can grab what we have for all entities and extend it with own methods, which are used to select item in inventory, drawing this inventory in fight phase and collecting items, when proceed on grid.

In [7]:
class Item:
    def __init__(self, name):
        self.coordinates = []
        self.name = name

    def set_coordinates(self, coordinates):
        self.coordinates = coordinates

    def get_symbol(self):
        return GRID_SYMBOLS[self.name]

    def get_coordinates(self):
        return self.coordinates

    def get_name(self):
        return self.name

**Item** class is representing item objects in game. It contains self coordinates, name of item and some service methods, used to get or set data of object.

In [8]:
class Weapon(Item):
    def __init__(self, weapon_name):
        self.name = weapon_name
        self.attack = WEAPONS[weapon_name]['attack']
        self.level = WEAPONS[weapon_name]['level']

    def base_attack(self):
        return self.attack

    def required_level(self):
        return self.required_level()

    def get_name(self):
        return self.name

    def get_icon(self):
        return WEAPONS_ICONS[self.name]

**Weapon** class is another one of class inheritance example. Basically, we can assume that weapon is an Item too, but we need some additional fields, like *attack* and *level* parameters. Different from Item class here is *get_icon* method, which used to get symbol representation, which after used to draw on game board.

In [9]:
class Fight:
    def __init__(self, monster, player):
        self.monster = monster
        self.player = player

    def monster_attack(self):
        attack_monster = self.monster.base_attack()
        self.player.get_hit(attack_monster)
        return attack_monster

    def player_attack(self):
        player_attack = self.player.base_attack()
        self.monster.get_hit(player_attack)
        return player_attack

    def draw_monster_and_inventory(self):
        cls()
        print(self.monster.get_name())
        print(MONSTER_IMAGE)
        print()
        print("'a' - Attack, 'c' - Change weapon")
        print()
        print('Monster HP: ', self.monster.hitpoints)
        print()
        print("Player's HP", self.player.hitpoints)
        self.player.draw_inventory()

    def draw_damage(self, player_damage, monster_damage):
        self.draw_monster_and_inventory()
        print()
        print('Your damage:', player_damage)
        print()
        print('Monster damage:', monster_damage)

**Fight** class defines the process of fight between 2 entities (player, monster). This object recieves *player*, and *monster* objects, to make method calls, to perform d fight process (player_attack, monster_attack) and to correctly draw fight process(monster name, image, entities health, players inventory, damage to each other)

In [10]:
def check_player_item(player_position, items):
    for item in items:
        if item.get_coordinates() == player_position:
            return item
        else:
            continue


def check_player_monster(player, monsters, grid):
    for monster in monsters:
        if monster.get_coordinates() == player.get_coordinates():
            fight_handler(player, monster, grid)
            break

After that, I've defined a few functions, which helps during the game process:
- *check_player_item*: checks, if player and item on the same position
- *check_player_monster*: checks, if player and monster on the same position

In [11]:
def fight_handler(player, monster, grid):
    fight = Fight(monster, player)
    fight.draw_monster_and_inventory()
    while 1:
        player_choice = input('Enter command: ')
        if player_choice == 'a':
            player_attack = fight.player_attack()
            if monster.get_health() <= 0:
                print('Monster is defeated!\nLevel Up!')
                grid.remove_monster(monster)
                input('Press Enter to continue: ')
                return
            monster_attack = fight.monster_attack()
            fight.draw_damage(player_attack, monster_attack)
            if player.get_health() <= 0:
                print('Game Over!')
                input('Press Enter to restart: ')
                game_start()
        elif player_choice == 'c':
            player.select_weapon()
            fight.draw_monster_and_inventory()
        else:
            print('\nWrong command!')
            fight.draw_monster_and_inventory()

*fight_handler()* function is one of flow control main functions, which is actually using objects methods, performing infinite while loop, to phase of fight, bewteen 2 entities (the condition of stop) is victory of player (monster health < 0) or victory of monster (player health < 0)

In [12]:
def game_handler(player, grid, items):
    while 1:
        grid.make_cells_visible(player.get_coordinates())
        grid.draw_grid()
        player_choise = input('Enter your move: ')
        if player_choise in MOVE_COMMANDS.keys() and \
                is_allowed_move(grid, (MOVE_COMMANDS[player_choise][0] + player.get_coordinates()[0],
                                       MOVE_COMMANDS[player_choise][1] + player.get_coordinates()[1])):
            player.move(MOVE_COMMANDS[player_choise])
            if player.get_coordinates() == list(grid.end_coordinates):
                if player.has_key():
                    cls()
                    print("Congratulations! You've completed demo dungeon")
                    return
                else:
                    player.move((-MOVE_COMMANDS[player_choise][0], -MOVE_COMMANDS[player_choise][1]))
                    print('No key')
                    continue
            grid.update_player(MOVE_COMMANDS[player_choise], player.get_coordinates())
            if item := check_player_item(player.get_coordinates(), items):
                player.add_weapon(item)
            check_player_monster(player, grid.get_monsters_list(), grid)
            grid.draw_grid()
        else:
            grid.draw_grid()
            print('Wrong move')
            continue

*game_handler()* too, one of the main functions. This func controls game board, player movement, ability to pickup items and start a fight phase, when **player** meets **monster**. It also build with infinite loop, where condition of exiting is to make to the exit of the dungeon (with key in inventory)

In [13]:
def game_start():
    game_board = Grid(10, 10)
    game_board.random_generator()
    player = Player('retroider', game_board.get_start_coordinates(), 30, attack=1, defence=2)
    game_board.update_player(player.coordinates, player.coordinates)
    game_items = [Weapon('knife'), Weapon('sword'), Weapon('axe'), Item('key')]
    game_board.place_items(game_items)
    player.add_weapon(Weapon('fists'))
    game_board.place_monsters()
    game_handler(player, game_board, game_items)
    print('Game Over!')
    return

*game_start()* function initializes game, for launch. It creates objects from classes (Grid, Weapon, Player, Entity), sets basic parameters, uses **Grid.random_generator** to build random dungeons and executes *game_handler*.

In [14]:
def is_allowed_move(grid, new_coord):
    return 0 <= new_coord[0] < grid.width and 0 <= new_coord[1] < grid.height and grid.get_element_symbol(
        new_coord) != '#'

And the last function in *is_allowed_move*. It basically checks, if player can make new move on the set game board and don't move away from board.

So, to execute overall code, we just call *game_start*

In [ ]:
game_start()

. . . . . # . . . . 
. . . . # . . . . . 
. . . . . . @ . . . 
. . . . # . . . . . 
. . . . . . . . . . 
. . . # . # . . . . 
. . . x . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 
. . . . . . . . . . 


wQuick note: honestly, to get a prefect code, I must use prefixes to fields of classes, to declare, that these fields and some of methods not public and must be used only from public methods of object, but I've forgot to do that, so it will be fairm if I lose some pts for this
P.S. Writing this at last, because this was implemented in last order. With method *Grid.make_cells_visible*, cells will become visible, if player is on distance with one cell (closest cells), so '.' symbol shows path, where ypu can go and cells, which are hidden from the player (I just didn't find better way to do it)